In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import mplfinance as mpf
from tensorflow import keras
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# NEW THINGS

In [2]:
stock_name = "AAPL"

In [3]:
def prepare_data(stock_name, period="5d", interval="1m",back_time=5):

    recent_data = yf.download(stock_name, period=period, interval=interval)
    data_200 = yf.download(stock_name, period=period, interval=interval).tail(len(recent_data) + 200)
    data_200 = data_200.drop(columns=["Adj Close"])

    data_200.fillna(method="bfill", inplace=True)

   # recent_data = data_200.tail(len(recent_data))

    recent_data = recent_data.iloc[:-1]

    for i in range(back_time):
        recent_data[f"Open t - {i+1}"] = recent_data["Open"].shift((i+1))
        recent_data[f"Close t - {i+1}"] = recent_data["Close"].shift((i+1))
        recent_data[f"High t - {i+1}"] = recent_data["High"].shift((i+1))
        recent_data[f"Low t - {i+1}"] = recent_data["Low"].shift((i+1))
    recent_data = recent_data.dropna()

    recent_data["movement"] = (recent_data["Close"] > recent_data["Open"]).astype(int)
    


       
    # add_plot = mpf.make_addplot(recent_data["200_day"], color="blue", linestyle="--")

    return recent_data

recent_data = prepare_data(stock_name,back_time=10,period="1mo",interval="1d")#data

recent_data.head()#sigma


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Open t - 1,Close t - 1,High t - 1,Low t - 1,...,Low t - 8,Open t - 9,Close t - 9,High t - 9,Low t - 9,Open t - 10,Close t - 10,High t - 10,Low t - 10,movement
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-24,224.000000,224.800003,217.130005,218.539993,218.539993,61777600,224.369995,225.009995,226.940002,222.679993,...,228.679993,231.389999,227.570007,232.389999,225.770004,229.300003,232.979996,233.080002,229.250000,0
2024-07-25,218.929993,220.850006,214.619995,217.490005,217.490005,51391200,224.000000,218.539993,224.800003,217.130005,...,233.089996,228.919998,230.539993,232.639999,228.679993,231.389999,227.570007,232.389999,225.770004,0
2024-07-26,218.699997,219.490005,216.009995,217.960007,217.960007,41601300,218.929993,217.490005,220.850006,214.619995,...,232.330002,236.479996,234.399994,237.229996,233.089996,228.919998,230.539993,232.639999,228.679993,0
2024-07-29,216.960007,219.300003,215.750000,218.240005,218.240005,36311800,218.699997,217.960007,219.490005,216.009995,...,226.639999,235.000000,234.820007,236.270004,232.330002,236.479996,234.399994,237.229996,233.089996,1
2024-07-30,219.190002,220.330002,216.119995,218.800003,218.800003,41643800,216.960007,218.240005,219.300003,215.750000,...,222.270004,229.449997,228.880005,231.460007,226.639999,235.000000,234.820007,236.270004,232.330002,0


In [57]:
from sklearn.model_selection import train_test_split
import torch
from torch import nn

#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

X = recent_data.drop(["Close", "Volume", "High", "Low","movement"], axis=1)
y = recent_data[["movement"]]

X_tensor = torch.from_numpy(X.values)
y_tensor = torch.from_numpy(y.values)


X_train,X_test,y_train,y_test = train_test_split(X_tensor,y_tensor,test_size=0.33)



X_train = X_train.to(torch.float32).to(device)
# X_time_step = [i for i in range(len(X_train))]
X_test= X_test.to(torch.float32).to(device)
y_train = y_train.to(torch.float32).to(device)
y_test =y_test.to(torch.float32).to(device)

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [20]:
class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=2):
        super(LSTMPredictor, self).__init__()

        self.ltsm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
        )
    def forward(self, sequences):
        lstm_out, (hn, cn) = self.ltsm(sequences)
        return lstm_out
    
model = nn.Sequential(
    
    LSTMPredictor(input_size=42, hidden_size=128, n_layers=10),
    nn.Linear(128, 1),
    
).to(device)

# model(X_train)

In [59]:
X_train

tensor([[219.1500, 219.8600, 224.3700, 218.3600, 224.4800, 217.0200, 221.4400,
         222.0800, 223.8200, 220.6300, 219.1900, 218.8000, 220.3300, 216.1200,
         216.9600, 218.2400, 219.3000, 215.7500, 218.7000, 217.9600, 219.4900,
         216.0100, 218.9300, 217.4900, 220.8500, 214.6200, 224.0000, 218.5400,
         224.8000, 217.1300, 224.3700, 225.0100, 226.9400, 222.6800, 227.0100,
         223.9600, 227.7800, 223.0900, 224.8200, 224.3100, 226.8000, 223.2800],
        [224.0000, 218.5400, 224.3700, 225.0100, 226.9400, 222.6800, 227.0100,
         223.9600, 227.7800, 223.0900, 224.8200, 224.3100, 226.8000, 223.2800,
         230.2800, 224.1800, 230.4400, 222.2700, 229.4500, 228.8800, 231.4600,
         226.6400, 235.0000, 234.8200, 236.2700, 232.3300, 236.4800, 234.4000,
         237.2300, 233.0900, 228.9200, 230.5400, 232.6400, 228.6800, 231.3900,
         227.5700, 232.3900, 225.7700, 229.3000, 232.9800, 233.0800, 229.2500],
        [206.9000, 209.8200, 205.3000, 207.2300, 2

In [106]:
epochs = 1000

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    model.train()

    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    model.eval()
    with torch.inference_mode():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred, y_test)
        if epoch % 100 == 0:
            print(f"epoch: {epoch} loss = {loss}, test loss = {test_loss}")

epoch: 0 loss = 0.002593838609755039, test loss = 890050.75
epoch: 100 loss = 0.0035256887786090374, test loss = 49.727272033691406
epoch: 200 loss = 0.0010608789743855596, test loss = 1889.29541015625
epoch: 300 loss = 0.0019466890953481197, test loss = 1353.6676025390625
epoch: 400 loss = 0.003622602904215455, test loss = 14050883.0
epoch: 500 loss = 0.0033486869651824236, test loss = 193.32086181640625
epoch: 600 loss = 0.0031670168973505497, test loss = 4284.4970703125
epoch: 700 loss = 0.0025408484507352114, test loss = 167.01861572265625
epoch: 800 loss = 0.004147311206907034, test loss = 483.46160888671875
epoch: 900 loss = 0.0020155487582087517, test loss = 52.639862060546875


In [ ]:
torch.save(model, "model1.pt")

In [60]:
def prepare_test_loop_data(stock_name, period="5d", interval="1m",back_time=5, answers =[]):

    recent_data = yf.download(stock_name, period=period, interval=interval)

    data_200 = yf.download(stock_name, period=period, interval=interval).tail(len(recent_data) + 201)
    data_200 = data_200.drop(columns=["Adj Close"])

    data_200.fillna(method="bfill", inplace=True)

    #recent_data = data_200.tail(len(recent_data))


    recent_data = recent_data.dropna()
    for i in range(back_time):
        recent_data[f"Open t - {i+1}"] = recent_data["Open"].shift((i+1))
        recent_data[f"Close t - {i+1}"] = recent_data["Close"].shift((i+1))
        recent_data[f"High t - {i+1}"] = recent_data["High"].shift((i+1))
        recent_data[f"Low t - {i+1}"] = recent_data["Low"].shift((i+1))
    recent_data["movement"] = (recent_data["Close"] > recent_data["Open"]).astype(int)

    # recent_data = recent_data.tail(1)
    # add_plot = mpf.make_addplot(recent_data["200_day"], color="blue", linestyle="--")

    return recent_data

#recent_data = prepare_test_loop_data(stock_name,back_time=1,answers=[answer])
#recent_data.head()


In [74]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

answers = []
forcast = 10
check_data = prepare_test_loop_data(
    interval="1m", back_time=10, period="1d", stock_name=stock_name
)
date_range = pd.date_range(
    start=check_data.index[-1], periods=forcast, freq="1T"
)  # Generate a date range



for i in range(forcast):
    # for i in answers:

    check_data_last = check_data.tail(1)
    check_X = check_data_last.drop(["Close", "Volume", "High", "Low","movement"], axis=1)
    check_y = check_data_last[["movement"]]

    # print(check_data.tail(6).to_markdown())
    # print(X.shape)

    check_X_tensor = torch.from_numpy(check_X.values)
    check_y_tensor = torch.from_numpy(check_y.values)

    check_X_tensor = check_X_tensor.to(torch.float32).to(device)
    check_y_tensor = check_y_tensor.to(torch.float32).to(device)

    # print(check_data_last.to_markdown(), "\n\n\n")

    answer = model(check_X_tensor)
    a = torch.softmax(answer, dim=1).argmax(dim=1)
    print(a)

    answer = answer.detach().cpu().numpy()

    answer = np.append(answer[0],check_X["Close t - 1"])
    answers.append(answer)
    df = pd.DataFrame(
        {
            "Close": [answers[-1][0]],
            "High": [answers[-1][1]],
            "Low": [answers[-1][2]],
            "Open": [answers[-1][3]],
            "Adj Close": [0],
            "Volume": [0],
        },
        index=[date_range[i]],
    )

    check_data = check_data._append(df)
    for i in range(10):
        check_data[f"Open t - {i+1}"] = check_data["Open"].shift((i + 1))
        check_data[f"Close t - {i+1}"] = check_data["Close"].shift((i + 1))
        check_data[f"High t - {i+1}"] = check_data["High"].shift((i + 1))
        check_data[f"Low t - {i+1}"] = check_data["Low"].shift((i + 1))
    recent_data["movement"] = (recent_data["Close"] > recent_data["Open"]).astype(int)

    # check_data.dropna(inplace=True)
# print(check_data.to_markdown())

# print(check_data)
# print(answers)

# sns.catplot(answer[0][:],label="Predicted")
last_elements = [arr[0] for arr in answers]

recent_data = prepare_data(stock_name, period="1d", interval="1m")
# print("Converted index to datetime")

answers_df = pd.concat(
    [
        pd.DataFrame(
            recent_data,
            columns=["Close", "High", "Low", "Open","movement"],
        ),
        pd.DataFrame(
            answers, columns=["movement"], index=date_range
        ),
    ]
).iloc[200:]

# print(answers_df.tail(100))

mpf.plot(
    answers_df,
    type="candle",
    style="charles",
    title=f"{stock_name} Candlestick Chart",
    ylabel="Price",
)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

tensor([0])


IndexError: index 2 is out of bounds for axis 0 with size 2

In [62]:
HIDDEN_SIZE = 64
EPOCHS = 20
BATCH_SIZE = 24
LEARNING_RATE = 0.001

In [64]:
from torch import nn

# Build model
class stonks(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):

        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
             nn.ReLU(), 
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features), 
            nn.Sigmoid(),
     
        )
    
    def forward(self, x):
        return self.linear_layer_stack(x)

# Create an instance of BlobModel and send it to the target device
model = stonks(input_features=X_train.size()[1], 
                    output_features=1, 
                    hidden_units=8).to(device)


In [65]:
y_train[0]

tensor([1.])

In [82]:
# Create loss and optimizer
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.1) # exercise: try changing the learning rate here and seeing what happens to the model's performance

In [86]:
# Make prediction logits with model
y_logits = model(X_test.to(device))

# Perform softmax calculation on logits across dimension 1 to get prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1)
print(y_logits[:5])
print(y_pred_probs[:5])

tensor([[2.1976e-05],
        [6.7180e-05],
        [2.2683e-05],
        [2.2327e-05]], grad_fn=<SliceBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SliceBackward0>)


In [85]:
# Fit the model
#torch.manual_seed(42)

# Set number of epochs
epochs = 1000

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass
    y_logits = model(X_train) # model outputs raw logits 
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
  
    loss = loss_fn(y_pred.to(torch.float32), y_train.squeeze()) 
  


    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.requires_grad = True
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test)


    # Print out what's happening
      if epoch % 100 == 0:
          print(f"Epoch: {epoch} | Loss: {loss:.5f},| Test Loss: {test_loss:.5f}") 

Epoch: 0 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 100 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 200 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 300 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 400 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 500 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 600 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 700 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 800 | Loss: 75.00000,| Test Loss: 2.40205
Epoch: 900 | Loss: 75.00000,| Test Loss: 2.40205
